## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [30]:
pwd

'd:\\AIHUB'

### Basic Function

In [2]:
def sorted_list(path_list):
    
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [3]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [4]:
def json_file_path_list(path_list):
    
    file_path  = [glob(i, recursive = True) for i in path_list][0]
    file_path = sorted_list(file_path)
    
    return file_path

In [5]:
def train_valid_json_file_path_list(path_list):

  train_file_path, valid_file_path = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_path = sorted_list(train_file_path)
  valid_file_path = sorted_list(valid_file_path)
    
  return train_file_path, valid_file_path

In [6]:
def txt_file_path_list(source_file_nested_list, folder_corpus_type_path):

  text_file_path_list = [folder_corpus_type_path + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_path_list

In [7]:
def make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list):

    train_file_path, valid_file_path = train_valid_json_file_path_list(json_path_list)
    
    the_number_of_json_file = len(train_file_path) + len(valid_file_path)
    print("The number of file:", the_number_of_json_file)
    
    train_text_file_path_list = txt_file_path_list(train_file_path, txt_path_list[0])
    valid_text_file_path_list = txt_file_path_list(valid_file_path, txt_path_list[1])

    return train_file_path, valid_file_path, train_text_file_path_list, valid_text_file_path_list

In [8]:
def make_json_txt_file_path_list(json_path_list, txt_path_list):

    file_path = json_file_path_list(json_path_list)
    
    the_number_of_json_file = len(file_path) 
    print("The number of file:", the_number_of_json_file)
    
    text_file_path_list = txt_file_path_list(file_path, txt_path_list[0])

    return file_path, text_file_path_list

In [9]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, " " + text + " ")    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, " " + text + " ")    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거  
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### AIHUB 논문자료 요약

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=90)

#### Convert JSON File to TXT File

In [10]:
json_path_list = ['AIHUB_논문자료 요약/Training/'+ '/**/*.json', 
                  'AIHUB_논문자료 요약/Validation/'+ '/**/*.json']
txt_path_list = ["exploration/summary_of_thesis_materials_pro/AIHUB_summary_of_thesis_materials_train_", 
                 "exploration/summary_of_thesis_materials_pro/AIHUB_summary_of_thesis_materials_valid_"]

In [11]:
train_file_list, valid_file_list, train_text_file_path_list, valid_text_file_path_list = \
    make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list)

The number of file: 65


In [12]:
source_file_index_df = pd.DataFrame(train_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/summary_of_thesis_materials_source_train_file_index.xlsx")

source_file_index_df = pd.DataFrame(valid_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/summary_of_thesis_materials_source_valid_file_index.xlsx")

In [13]:
def make_source_list(source_file, json_sample):
    
    source_df = pd.DataFrame(json_sample['data'])

    if '논문/논문요약' or '특허섹션' in source_file:
            
        try:
            summary_entire_source_dict = dict(source_df['summary_entire'].explode())
            summary_entire_source_json = pd.json_normalize(summary_entire_source_dict)
            summary_entire_source_list = list(summary_entire_source_json.filter(regex='orginal_text').values[0])

            summary_section_source_dict = dict(source_df['summary_section'].explode())
            summary_section_source_json = pd.json_normalize(summary_section_source_dict)
            summary_section_source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

            source_list = summary_entire_source_list + summary_section_source_list

        except:
            summary_section_source_dict = dict(source_df['summary_section'].explode())
            summary_section_source_json = pd.json_normalize(summary_section_source_dict)
            source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])
                
    else:
        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

    return source_list

In [19]:
def count_number_of_txt_file_with_batch_list(source_file_list, batch_size):
    
    source_file_by_batch_df = pd.DataFrame({'File':[0], 'Length of Source List':[0],
                                            'The Number of txt File':[0], 
                                            'Description':[0]})
                                            
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    
    for i in range(len(source_file_list)):    
        
        source_file = source_file_list[i]        

        with open(source_file, 'r', encoding='utf-8') as one_json_file:
            one_json_sample = json.load(one_json_file) 

        source_list = make_source_list(source_file, one_json_sample)
        
        the_number_of_txt_file = ((len(source_list) // batch_size) + 1) 

        if len(source_list) >= batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file, ""]
            the_number_of_txt_file_list.append(the_number_of_txt_file)
            the_number_of_total_txt_file  += the_number_of_txt_file

        elif len(source_list) < batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file,
                                              "not subject of batch. small source list."]
            the_number_of_txt_file_list.append(1)
            the_number_of_total_txt_file  += 1

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    if 'rain' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/summary_of_thesis_materials_train.xlsx")
    elif 'alid' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/summary_of_thesis_materials_valid.xlsx")
    else:
         source_file_by_batch_df.to_excel("source_file_by_batch/summary_of_thesis_materials.xlsx")
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [20]:
def write_jsontext_to_txt_file_with_batch_list(source_file_list, text_file_path_list, batch_size, the_number_of_txt_file_list):

  progress_length = sum(the_number_of_txt_file_list)
  print("[Size]")
  print("The number of preprocessing corpus: " + str(progress_length))
  print("\n[Order]")
  pbar = tqdm(range(progress_length))
  num = 0
  
  for i in range(len(source_file_list)):

    source_file = source_file_list[i]
    
    with open(source_file, 'r', encoding='utf-8') as one_json_file:
      one_json_sample = json.load(one_json_file)

    source_list = make_source_list(source_file, one_json_sample)
    
    n = batch_size
    source_batch_list = list(divide_source_file_list(source_list, n))
      
    for source_list in source_batch_list:   
      with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:
        fp.write("\n".join(source_list))           
      num += 1  
      pbar.n += 1
      pbar.refresh()
      time.sleep(0.01)
  pbar.close()  

In [21]:
batch_size = 1000
the_number_of_train_txt_file, the_number_of_train_txt_file_list = count_number_of_txt_file_with_batch_list(train_file_list, batch_size)

Batch Size: 1000
The number of txt file: 726


In [22]:
source_file_by_batch_train_df = pd.read_excel('source_file_by_batch/summary_of_thesis_materials_train.xlsx', engine='openpyxl')  
source_file_by_batch_train_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_논문자료 요약/Training\training_특허섹션만\9-N-0.js...,38000,39,NaN
1,1,AIHUB_논문자료 요약/Training\training_특허섹션만\9-N-1.js...,38000,39,NaN
2,2,AIHUB_논문자료 요약/Training\training_특허섹션만\10-N-0.j...,38000,39,NaN
3,3,AIHUB_논문자료 요약/Training\training_논문\논문요약_0206_0...,64000,65,NaN
4,4,AIHUB_논문자료 요약/Training\training_논문\논문요약_0206_1...,64000,65,NaN
5,5,AIHUB_논문자료 요약/Training\training_논문\논문요약_0206_2...,12518,13,NaN
6,6,AIHUB_논문자료 요약/Training\training_논문\논문요약_0220_0...,20042,21,NaN
7,7,AIHUB_논문자료 요약/Training\training_특허전체\특허전체_9-Y-...,6000,7,NaN
8,8,AIHUB_논문자료 요약/Training\training_특허전체\특허전체_9-Y-...,6000,7,NaN
9,9,AIHUB_논문자료 요약/Training\training_특허전체\특허전체_9-Y-...,6000,7,NaN


In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list, train_text_file_path_list,
                batch_size, the_number_of_train_txt_file_list)

In [23]:
batch_size = 1000
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = count_number_of_txt_file_with_batch_list(valid_file_list, batch_size)

Batch Size: 1000
The number of txt file: 117


In [26]:
source_file_by_batch_valid_df = pd.read_excel('source_file_by_batch/summary_of_thesis_materials_valid.xlsx', engine='openpyxl')  
source_file_by_batch_valid_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_논문자료 요약/Validation\10-N-1.json ~ AIHUB...,37000,38,NaN
1,1,AIHUB_논문자료 요약/Validation\validation_논문\논문요약_02...,23604,24,NaN
2,2,AIHUB_논문자료 요약/Validation\validation_논문\논문요약_02...,12518,13,NaN
3,3,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
4,4,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
5,5,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
6,6,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
7,7,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN
8,8,AIHUB_논문자료 요약/Validation\validation_특허전체\특허전체_...,6000,7,NaN


In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list, valid_text_file_path_list,
                batch_size, the_number_of_valid_txt_file_list)

#### Preprocess TXT File

In [ ]:
def post_txt_file_path_list(corpus_path_list):
   
  post_corpus_path_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_path_list]

  return post_corpus_path_list

In [ ]:
def make_pro_post_txt_file_path_list(pro_corpus_path):
    
    pro_total_corpus_path_list = glob(pro_corpus_path)
    pro_total_corpus_path_list = sorted_list(pro_total_corpus_path_list)
    post_total_corpus_path_list = post_txt_file_path_list(pro_total_corpus_path_list)

    return pro_total_corpus_path_list, post_total_corpus_path_list

In [ ]:
pro_corpus_path = "AIHUB_corpus/exploration/summary_of_thesis_materials_pro/AIHUB_summary_of_thesis_materials_" + "*.txt"
make_pro_post_txt_file_path_list(pro_corpus_path)

In [ ]:
len(pro_total_corpus_path_list)

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

In [ ]:
ray.init(num_cpus = 4)

@ray.remotes
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, " " + text + " ")    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, " " + text + " ")    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)
                
    return preprocessing_sentence_list

In [ ]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):

    progress_length = len(pro_total_corpus_path_list)   
    print("[Size]")
    print("The number of preprocessing corpus: " + str(progress_length))
    print("\n[Order]")
    pbar = tqdm(range(progress_length))
    num = 0
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))
        
        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))
        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)

    pbar.close() 

In [ ]:
pro_total_corpus_list, post_total_corpus_list = preprocessing_corpus_txt(pro_total_corpus_list, post_total_corpus_list)

In [ ]:
ray.shutdown()

In [ ]:
def merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path,
                                      deduplicate_corpus_path):
    
    corpus_list = glob(preprocessing_corpus_path)
    corpus_list = sorted_list(corpus_list)
    
    with open(merge_corpus_path, 'w', encoding='utf-8') as f:
        for corpus in corpus_list:
            with open(corpus, encoding='utf-8') as text:
                for line in text:
                    f.write(line)
                    
    with open(deduplicate_corpus_path, 'w', encoding='utf-8') as f1:
        with open(merge_corpus_path, encoding='utf-8') as f2:
            lines = f2.read().splitlines()
            single_sentence_dict = dict.fromkeys(lines)
            single_sentence_list = list(single_sentence_dict)
            f1.write("\n".join(single_sentence_list))                

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/summary_of_thesis_materials_post/AIHUB_summary_of_thesis_materials_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_summary_of_thesis_materials.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_summary_of_thesis_materials.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)